# WORKING

### How FAISS Works

- #### FAISS provides various methods to store and retrieve embeddings efficiently. The most common technique is IndexFlatL2, which performs a brute-force L2 (Euclidean) distance search efficiently.

#### 👉 Basic Steps:

- #### Convert Text into Embeddings:
    - #### Sentences like "I love pizza" are converted into vectors using an embedding model (e.g., OpenAI's text-embedding-ada-002 or SentenceTransformers from Hugging Face).
    - #### Example: "I love pizza" → [0.4, 0.6, 0.1, 0.8]

- #### Store Embeddings in FAISS
    - #### FAISS stores these embeddings in a searchable index.

- #### Query Search
    ##### When you search with "Pizza is amazing", its embedding is computed and compared against stored embeddings using a distance metric (like cosine similarity or Euclidean distance).

In [2]:
!pip install faiss-cpu   # or faiss-gpu if you have a GPU


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 28.2 MB/s eta 0:00:0000:0100:01


In [3]:
import faiss
import numpy as np

# Example document embeddings (each row represents a sentence embedding)
document_vectors = np.array([
    [0.1, 0.8, 0.3],  # Embedding for "I love pizza"
    [0.5, 0.2, 0.9],  # Embedding for "Pizza is my favorite"
    [0.3, 0.7, 0.2]   # Embedding for another document
]).astype("float32")

# Create FAISS index using L2 distance
index = faiss.IndexFlatL2(3)  # 3D vector space
index.add(document_vectors)  # Add documents to index

# Query vector (embedding for "I enjoy eating pizza")
query_vector = np.array([[0.2, 0.75, 0.25]]).astype("float32")

# Search for the closest document
_, indices = index.search(query_vector, 1)  # Find 1 closest match

print(f"Most similar document index: {indices[0][0]}")


Most similar document index: 2


## Let's work with a basic Q/A dataset to build a basic Q/A chatbot prototype

In [4]:
customer_queries = {
"How do I place an order?": "To place an order, browse products, add them to your cart, and proceed to checkout. Enter your shipping and payment details and confirm the order.",

"Can I modify my order after placing it?": "You can modify your order within 30 minutes of placing it by going to 'My Orders' and selecting 'Edit Order'. After this period, modifications may not be possible.",

"How do I cancel my order?": "Go to 'My Orders', select the order, and click 'Cancel'. If the order has already been shipped, you may need to request a return instead.",

"What payment methods do you accept?": "We accept credit/debit cards, UPI, net banking, PayPal, and cash on delivery (COD) for eligible orders.",

"How do I track my order?": "You can track your order by clicking on 'My Orders' and selecting 'Track Order'. You’ll receive a tracking link via email/SMS once it’s shipped.",

"What is your return policy?": "You can return products within 7 days of delivery if they are unused and in original packaging. Refunds are processed within 5-7 business days.",

"How do I initiate a return?": "Go to 'My Orders', select the order, and click on 'Return'. Follow the instructions to schedule a pickup or drop-off.",

"When will I get my refund?": "Refunds for prepaid orders are processed within 5-7 business days after the return is approved. COD orders receive a refund via bank transfer or store credit.",

"Why is my order delayed?": "Order delays can happen due to high demand, weather conditions, or courier service issues. Check 'My Orders' for real-time tracking updates.",

"Do you offer express delivery?": "Yes, we offer express delivery in select locations for an additional fee. Choose 'Express Shipping' at checkout if available.",

"Can I change my delivery address?": "You can change the delivery address before the order is shipped by going to 'My Orders' and selecting 'Edit Address'. Once shipped, address changes are not possible.",

"What should I do if I receive a damaged product?": "If you receive a damaged product, report it within 48 hours by visiting 'My Orders' and selecting 'Report Issue'. You may be eligible for a replacement or refund.",

"Do you provide international shipping?": "Currently, we ship only within [Country Name]. International shipping options will be available soon.",

"How do I apply a discount code?": "Enter your discount code at checkout in the 'Apply Coupon' section. If valid, the discount will be applied to your total amount.",

"Why was my payment declined?": "Payments may be declined due to incorrect details, insufficient funds, or bank restrictions. Try using another payment method or contact your bank.",

"Can I buy now and pay later?": "Yes, we offer 'Buy Now, Pay Later' options through [BNPL Provider]. Choose this option at checkout to pay in installments.",

"How can I contact customer support?": "You can reach our customer support via email at support@example.com, live chat on our website, or by calling our helpline at [phone number].",
}


In [5]:
questions=[i for i in customer_queries.keys()] # Get all the keys from the dictionary(questions)

# get the maximum length of sentences among all in questions
- ## helpful to get padding of vector embedding (better for visualizing)

In [6]:
from transformers import AutoTokenizer
t = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenized_sentences = [t.tokenize(se) for se in questions]
max_length = max(len(tokens) for tokens in tokenized_sentences)
max_length

/workspaces/langchain_ai_bot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


11

### Encode questions to vector-vector Embedding

In [7]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


# You can do it using library 
# Embeddings = model.encode(questions, 
#                           convert_to_numpy=True,  # Get NumPy output
#                           normalize_embeddings=True,  # Normalize the output
#                             show_progress_bar=True)  # Show progress

## If you want to do it manually
encoded_questions = model.encode(questions, 
                           convert_to_numpy=True, 
                            normalize_embeddings=True ) # Normalize for better similarity (some tokens might have higher value commpared to others in vector space so we normalize it)
encoded_questions


array([[-0.00525271, -0.00277411,  0.00439008, ...,  0.05966293,
        -0.00959073, -0.03116939],
       [-0.01255482,  0.02828169,  0.03745552, ..., -0.05449414,
         0.0244373 , -0.05462256],
       [-0.0098854 ,  0.059414  ,  0.07196154, ..., -0.00860798,
        -0.04153501, -0.06529608],
       ...,
       [ 0.03389939,  0.11285318,  0.09194537, ..., -0.06230308,
        -0.03781408, -0.02075071],
       [-0.01548824, -0.00544531, -0.00193918, ..., -0.09687743,
         0.07747193, -0.05970661],
       [-0.06125688,  0.0057992 ,  0.05149609, ..., -0.00036166,
        -0.00158964, -0.00253615]], shape=(17, 384), dtype=float32)

# Provide questions from user and finding similarity with given questions in our dataset to fetch answer

## Fetching question from user and encoding it in vector

In [9]:
user_questions=input("Enter Your question: ") # Get the user question
encoded_user_questions= model.encode(user_questions, 
                           padding='max_length',  
                           truncation=True, 
                           max_length=max_length, 
                           return_tensors="pt").reshape(1,-1)   # cosine_similarity() requires 2D input. So, reshaping it to 2D
encoded_user_questions


array([[-4.42839824e-02,  5.39841689e-02,  1.00228071e-01,
         3.79077978e-02, -5.60873076e-02, -5.90652041e-03,
         5.44565506e-02, -7.84180835e-02,  9.31056142e-02,
         6.50185645e-02,  5.15615307e-02,  6.06659427e-02,
        -1.61820650e-02, -8.19350258e-02, -1.07381471e-01,
        -2.72736251e-02, -8.57131556e-03,  5.02477475e-02,
        -7.20677748e-02,  1.05813025e-02, -3.60664092e-02,
         7.78846219e-02, -9.57318302e-03,  4.91333604e-02,
         7.72515982e-02, -1.32186562e-02, -7.67742172e-02,
         4.55384552e-02, -2.08106376e-02, -7.61239678e-02,
         6.65588130e-04,  1.26025686e-02,  7.84345493e-02,
        -1.88098382e-02, -4.62053679e-02, -5.81834130e-02,
        -4.96258624e-02, -5.81564419e-02,  3.59378681e-02,
        -2.83714645e-02, -2.12047230e-02, -1.16445206e-03,
        -2.72603761e-02, -7.33287409e-02, -6.22065738e-03,
         1.56951230e-02, -4.44557443e-02,  1.62821170e-02,
         8.09194371e-02,  6.73385113e-02,  2.63451543e-0

# Cosine Similarity

In [11]:

from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Compute similarity with all   query in our dataset
similarity = cosine_similarity(encoded_user_questions,encoded_questions)
most_similar_idx = np.argmax(similarity)
print(f"Cosine Similarity: {similarity[0][0]:.2f}")
print(f'Question that matches it is : {questions[most_similar_idx]}')
print(f'Question that matches it is : {questions[most_similar_idx]}\n Answer to it is: {(customer_queries[questions[most_similar_idx]]).replace('.','.\n')}')

Cosine Similarity: 0.45
Question that matches it is : How do I cancel my order?
Question that matches it is : How do I cancel my order?
 Answer to it is: Go to 'My Orders', select the order, and click 'Cancel'.
 If the order has already been shipped, you may need to request a return instead.



<!-- How FAISS Works

FAISS provides various methods to store and retrieve embeddings efficiently. The most common technique is IndexFlatL2, which performs a brute-force L2 (Euclidean) distance search efficiently.

👉 Basic Steps:

    Convert Text into Embeddings
        Sentences like "I love pizza" are converted into vectors using an embedding model (e.g., OpenAI's text-embedding-ada-002 or SentenceTransformers from Hugging Face).
        Example: "I love pizza" → [0.4, 0.6, 0.1, 0.8]

    Store Embeddings in FAISS
        FAISS stores these embeddings in a searchable index.

    Query Search
        When you search with "Pizza is amazing", its embedding is computed and compared against stored embeddings using a distance metric (like cosine similarity or Euclidean distance). -->